# Statistical Hypothesis Testing
- 모집단의 모수에 관한 가설을 세우고 표본의 정보를 이용해 그 가설을 검증하는 기법

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.formula.api as smf

%precision 3
%matplotlib inline

In [ ]:
# To check the weight of fries over 130g 
df = pd.read_csv('../data/ch11_potato.csv')
sample = np.array(df['무게']) #Weight
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [5]:
s_mean = np.mean(sample)
s_mean

128.451

## 11.1 Statistical Hypothesis Testing

### 1.1.1 flow of hypothesis testing

In [ ]:
# 1)  
rv = stats.norm(130, np.sqrt(9/14))
rv.isf(0.95) # 표본 평균이 128.681 이하의 무게가 되는 것이 5퍼센트 이하의 확률로 발생한다.

# 2) 모평균이 130인게 맞아? 더 적은 숫자 아닐까?

128.681

- 귀무가설 (null hypothesis) : 차이가 있다, 효과가 있다  H0
- 대립가설 (alternative hypothesis) : 차이가 없다, 효과가 없다 H1
######
- reject the null hypothesis : the n.h is wrong, significant  
- accpet the null hypothesis : Can't say the n.h is wrong (=/= n.h is right)
######
- rejection region
- acceptance region
#####
- 기각역에 들어가는 확률을 정하고 검정을 수행해야한다. : 유의 수준 (level of significant) 5%
- 경계선상의 값을 임곗값 critical value 128.681
- 검정에 사용되는 통계값을 검정 통계량 test statistic sample value
####
- p_value < 유의수준 : reject null hypothesis vs accept


In [ ]:
# Test statistic sample value (standalization)
z = (s_mean - 130) / np.sqrt(9/14)
z


-1.932

In [ ]:
# Critical value
rv = stats.norm()
rv.isf(0.95)

# Reject n.h and sample mean is lower than 130 g

-1.645

In [12]:
# Check the p 
rv.cdf(z)

0.027

### 11.1.2. One-tailed test & two-tailed test
- 모평균은 130g 이 아니다 라는 대립가설로 가설검정을 수행할 때도 있다
- 이의 경우에 130g보다 큰 경우도 고려하는데 이를 two-tailed test 라고 한다
- 단측 검정 측의 기각역이 더 넓다  : 양측 검정 보다 귀무가설을 기각하기 쉽다

In [13]:
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932

In [14]:
rv = stats.norm()
rv.interval(0.95)

(-1.960, 1.960)

In [15]:
# Two side → two times 
rv.cdf(z) * 2

0.053

### 11.1.3. Two errors
- 제 1종 오류 : 귀무가설이 옳을 때, 귀무가설을 기각하는 오류 - 위험률         - 분석가 제어 가능
- 제 2종 오류 : 대립가설일 옳을 때, 귀무가설을 채택하는 오류 - 검정력 power   - 분석가 제어 불가

In [ ]:
# 1종 오류 : 모평균이 130g인데도 평균은 130g보다 작다에 도달하는 경우 
# 모집단의 확률 분포
rv = stats.norm(130, 3)

In [ ]:
c = stats.norm().isf(0.95)
n_samples = 10000                     # 2) Testing 10000 times
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2) # 1) Select 14 sample
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z < c:
        cnt += 1
cnt / n_samples

# False positive : 0.053

0.053

In [21]:
# 2종 오류 : 모평균이 130g 보다 작다 인데도, 이 결론을 얻을 수 없는 상황 (미탐 false negative)
rv = stats.norm(128, 3)

c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z >= c:
        cnt += 1
        
cnt / n_samples

0.195